### Id matching using SQL redirect table

In [ ]:
import re
from urllib.parse import unquote
import dask.dataframe as dd
import pandas


annotation_ids = dd.read_parquet('annotation_ids.parquet', engine='pyarrow')
doc_ids = dd.read_parquet('doc_ids.parquet', engine='pyarrow')

assert len(annotation_ids['ids']) == len(annotation_ids['ids'].unique())

In [7]:
# preprocessing
annotation_ids['ids'] = annotation_ids['ids'].map(lambda x: re.sub('https://en.wikipedia.org/wiki/', '', x))
doc_ids['ids'] = doc_ids['ids'].map(lambda x: re.sub('https://en.wikipedia.org/wiki/', '', x))

# get matches that don't require sql redirect lookup
matched_ids = annotation_ids.merge(doc_ids, how='inner', on='ids')
matched_ids['matched_ids'] = matched_ids['ids'] 
f'{len(matched_ids)}/{len(annotation_ids)}'

'2958950/5453207'

In [8]:
unmatched_ids = annotation_ids[~annotation_ids['ids'].compute().isin(matched_ids['ids'])]

In [9]:
unmatched_ids.compute()[:3]

,ids
1,1952%20chinese%20census
7,edward%20costello%20%28queensland%20politician%29
8,u.s.%20department%20of%20defense


In [10]:
import urllib.parse


unmatched_ids['ids'] = unmatched_ids['ids'].map(lambda x: urllib.parse.unquote(re.sub('%20', '_', x))) # url decode but whitespaces are underscores
doc_ids['ids'] = doc_ids['ids'].map(lambda x: urllib.parse.unquote(re.sub('%20', '_', x)))

unmatched_ids.compute()[:3]

,ids
1,1952_chinese_census
7,edward_costello_(queensland_politician)
8,u.s._department_of_defense


In [11]:
matched_ids = matched_ids.compute()
unmatched_ids = unmatched_ids.compute()
doc_ids = doc_ids.compute()

In [22]:
import mysql.connector


matches = []

def add_matches(chunk):
    connection = mysql.connector.connect(host='10.107.229.70',
                                        database='wikidata',
                                        user='user',
                                        password='password',
                                        buffered=True)
    with connection.cursor() as cursor:    
        print(chunk)
        query = f"select page_id, page_title from page where page_namespace = 0 and page_is_redirect = 1 and lower(convert(page_title using latin1)) in {tuple(chunk)}"
        cursor.execute(query)
        pages = cursor.fetchall()
        for page in pages:
            page_id, page_title = page
            page_title = page_title.decode('utf-8')
            
            redirect_query = f'select rd_title from redirect where rd_from = {page_id};'
            cursor.execute(redirect_query)
            redirect = cursor.fetchone()
            redirect = redirect[0].decode('utf-8')
            matches.append({'ids': page_title.lower(), 'matched_ids': redirect})    

#### TODO: scale matching
This can be achieved by either increasing SQL query tuple (`... in ()`) or using multithreading / -processing in some form.

#### TODO: case sensitive parsing
Matched ids should be actual working links. 

In [23]:
# def chunks(l, n):
    # n = max(1, n)
    # return (l[i:i+n] for i in range(0, len(l), n))
# unmatched = chunks(unmatched_ids, len(unmatched_ids['ids'])//20000)

# threads = []
# for chunk in list(unmatched):
    # chunk = chunk['ids']    
    # thread = threading.Thread(target=add_matches, args=(chunk,))
    # threads.append(thread)

# for thread in threads:
    # thread.start()

# for thread in threads:
    # thread.join()

add_matches(unmatched_ids['ids'][:100])

1                          1952_chinese_census
7      edward_costello_(queensland_politician)
8                   u.s._department_of_defense
9      2009_nba_all-star_game#rookie_challenge
14                                shaneeshwara
                        ...                   
216                            henry_c._pelton
222                             zipaquira_line
223                list_of_rc_relizane_seasons
224                                 ultrasharp
227                               at-the-money
Name: ids, Length: 100, dtype: object


In [24]:
len(matches)

51

In [26]:
matched_ids.append(matches)

,ids,matched_ids
0,chris%20chantler%20%28footballer%29,chris%20chantler%20%28footballer%29
1,national%20archives%20administration%20of%20china,national%20archives%20administration%20of%20china
2,aftab%20seth,aftab%20seth
3,kerstin%20stegemann,kerstin%20stegemann
4,jerome%20lewis,jerome%20lewis
...,...,...
46,1952_chinese_census,First_National_Population_Census_of_the_People...
47,trans-european_transport_networks,Trans-European_Transport_Networks
48,riemann_integrability,Riemann_integral
49,my_full_moon,Scrubs_(season_8)


For this batch of unmatched ids around 50% could be matched using SQL redirect table lookup